In [ ]:
import json
import re
import nltk
import os
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# 下载必要资源
# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# MBTI 类型关键词（大写匹配）
mbti_types = {
    'INTJ', 'INTP', 'ENTJ', 'ENTP',
    'INFJ', 'INFP', 'ENFJ', 'ENFP',
    'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ',
    'ISTP', 'ISFP', 'ESTP', 'ESFP'
}

# 深度清洗：用于 posts_cleaned
def clean_text(text):
    text = text.lower()
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = text.replace('|', ' ')
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = nltk.word_tokenize(text)
    cleaned = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return ' '.join(cleaned)

# 轻度清洗：用于 post_casebank（仅去链接）
def clean_casebank_keep_format(text):
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# 主清洗逻辑
def preprocess_mbti_json(input_json_path, output_json_path=None, output_csv_path=None):
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data = []
    MIN_TOKEN_LEN = 10  # 词数下限

    for item in tqdm(data, desc="🧹 Cleaning MBTI posts"):
        raw_posts = item.get("posts", "")
        segments = raw_posts.strip("'").split("|||")

        cleaned_segments = []
        casebank_segments = []

        for seg in segments:
            seg = seg.strip()
            has_url = 'http' in seg or 'www.' in seg
            token_len = len(nltk.word_tokenize(seg))
            contains_mbti = any(t in seg.upper() for t in mbti_types)

            # 丢弃规则统一：短 + URL + MBTI 标签 → 丢
            keep = not has_url and token_len > MIN_TOKEN_LEN and not contains_mbti

            if keep:
                cleaned = clean_text(seg)
                casebank_cleaned = clean_casebank_keep_format(seg)

                if cleaned:
                    cleaned_segments.append(cleaned)
                if casebank_cleaned:
                    casebank_segments.append(casebank_cleaned)

        cleaned_data.append({
            "type": item.get("type"),
            "posts": raw_posts,
            "posts_cleaned":item.get("posts_cleaned"),
            "post_casebank": " ".join(casebank_segments)
        })

    # 保存 JSON
    if output_json_path:
        os.makedirs(os.path.dirname(output_json_path) or ".", exist_ok=True)
        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(cleaned_data, f, ensure_ascii=False, indent=2)
        print(f"✅ JSON saved to: {output_json_path}")

    # 提示 CSV 输出
    if output_csv_path:
        print("⚠️ CSV建议只导出字段：type, posts_cleaned, post_casebank")


In [ ]:
preprocess_mbti_json(
    input_json_path="extended_mbti_dataset_v17.json",
    output_csv_path="extended_mbti_v17_cleaned.csv",
    output_json_path="casebank_mbti.json"
)


In [ ]:
#lora 清理部分
import json, re, hashlib
from collections import Counter
# from typing import Optional  # 如果是 py3.8/3.9，打开并把下面的返回类型改成 Optional[dict]

MBTI_16 = {"INTJ","INTP","ENTJ","ENTP","INFJ","INFP","ENFJ","ENFP",
           "ISTJ","ISFJ","ESTJ","ESFJ","ISTP","ISFP","ESTP","ESFP"}

def light_clean(x: str) -> str:
    if not x: return ""
    x = re.sub(r'(https?://\S+|www\.\S+)', ' ', x)
    x = x.replace('|',' ')
    x = re.sub(r'\s+', ' ', x).strip()
    return x

def build_case_row(r: dict, max_chars: int = 1200) -> dict:  # 如果低版本，写 Optional[dict]
    t = str(r.get("type","")).upper().strip()
    if t not in MBTI_16: 
        return None

    base = light_clean(r.get("posts_cleaned") or r.get("posts") or "")
    sem  = light_clean(r.get("semantic_view",""))
    sen  = light_clean(r.get("sentiment_view",""))
    lin  = light_clean(r.get("linguistic_view",""))

    if not base:
        return None

    meta = []
    if sem: meta.append(f"[Semantic] {sem}")
    if sen: meta.append(f"[Sentiment] {sen}")
    if lin: meta.append(f"[Linguistic] {lin}")
    post_casebank = base if not meta else f"{base} " + " ".join(meta)

    if len(post_casebank) > max_chars:
        post_casebank = post_casebank[:max_chars].rstrip() + "…"

    return {"type": t, "post_casebank": post_casebank, "embed_text": base}

def make_casebank(in_file: str, out_file: str, max_chars: int = 1200, dedup: bool = True):
    data = json.load(open(in_file, "r", encoding="utf-8"))
    out, seen = [], set()

    for r in data:
        row = build_case_row(r, max_chars=max_chars)
        if not row: 
            continue
        if dedup:
            h = hashlib.md5(row["embed_text"].encode("utf-8")).hexdigest()
            if h in seen:
                continue
            seen.add(h)
        out.append(row)

    json.dump(out, open(out_file, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

    cnt = Counter([x["type"] for x in out])
    total = len(out)
    print(f"✅ Casebank 完成：{total} 条 → {out_file}")
    print("📊 类型分布：")
    for k in sorted(cnt):
        print(f"  {k}: {cnt[k]} ({cnt[k]/total:.2%})")


In [ ]:
INPUT_PATH  = "pandora_eval_80.json"              # 你的文件
OUTPUT_PATH = "casebank_pandora_eval_80.json"     # 输出

make_casebank(INPUT_PATH, OUTPUT_PATH, max_chars=1200, dedup=True)


In [ ]:
import os
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from peft import PeftModel

BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_DIR = "mbti_lora_qwen1.5b-split_kaggle_ckpt"   # ✅ 指向包含 adapter_config.json 的目录

assert os.path.exists(os.path.join(ADAPTER_DIR, "adapter_config.json")), "路径不对，找不到 adapter_config.json"

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

cfg = AutoConfig.from_pretrained(BASE_MODEL, trust_remote_code=True)
cfg.num_labels = 16

base = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, config=cfg, trust_remote_code=True,
    device_map={"": "cuda:0"}, low_cpu_mem_usage=True
)

model = PeftModel.from_pretrained(base, ADAPTER_DIR, is_trainable=False).eval()
print("✅ LoRA 适配器加载成功")


In [ ]:
# extract_embeddings_qwen_meanpool.py
# -*- coding: utf-8 -*-
import json
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from peft import PeftModel

# ===== 路径配置 =====
BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_DIR = "mbti_lora_qwen1.5b-split_kaggle_ckpt"
INPUT_FILE  = "train.json"
OUTPUT_FILE = "casebank_A_train_80_with_embeddings.json"

# 设备
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda:0" if use_cuda else "cpu")
device_map = {"": "cuda:0"} if use_cuda else None

# ===== tokenizer & model =====
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config = AutoConfig.from_pretrained(BASE_MODEL, trust_remote_code=True)
config.num_labels = 16

base = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    config=config,
    trust_remote_code=True,
    device_map=device_map,
    low_cpu_mem_usage=True,
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR, is_trainable=False)
model.eval().to(device)

# ===== 句向量提取（masked mean pooling；推荐） =====
@torch.no_grad()
def get_text_embedding(text: str, max_len: int = 512) -> list[float]:
    ins = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_len
    ).to(device)

    out = model.base_model.model(
        **ins, output_hidden_states=True, use_cache=False, return_dict=True
    )
    last_hidden = out.hidden_states[-1]          # (B, T, H)
    mask = ins["attention_mask"].unsqueeze(-1)   # (B, T, 1)

    # masked mean pooling
    masked = last_hidden * mask                  # zero out pads
    sum_hidden = masked.sum(dim=1)               # (B, H)
    lengths = mask.sum(dim=1).clamp(min=1)       # (B, 1)
    emb = sum_hidden / lengths                   # (B, H)

    # L2 归一化（和 FAISS 里一致：余弦=内积）
    emb = F.normalize(emb, p=2, dim=1)           # (B, H)
    return emb.squeeze(0).cpu().tolist()

# （可选）最后 token pooling 版本：
# def get_text_embedding(text: str, max_len: int = 512) -> list[float]:
#     ins = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_len).to(device)
#     out = model.base_model.model(**ins, output_hidden_states=True, use_cache=False, return_dict=True)
#     last_hidden = out.hidden_states[-1]                # (B, T, H)
#     idx = ins["attention_mask"].sum(dim=1) - 1         # (B,)
#     emb = last_hidden[torch.arange(last_hidden.size(0)), idx]  # (B, H)
#     emb = F.normalize(emb, p=2, dim=1)
#     return emb.squeeze(0).cpu().tolist()

# ===== 处理 casebank =====
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

output = []
for item in tqdm(data, desc="Extracting sentence embeddings"):
    # 仍使用你构建 casebank 时的展示字段
    text = item.get("post_casebank") or item.get("embed_text") or item.get("posts_cleaned") or item.get("posts") or ""
    if not text.strip():
        continue
    emb = get_text_embedding(text)
    item["embedding"] = emb
    output.append(item)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print(f"✅ 完成！结果已保存到 {OUTPUT_FILE}")


In [ ]:
# build_faiss_from_casebank.py
# -*- coding: utf-8 -*-
import json, re
import faiss
import numpy as np

DATA_PATH       = "casebank_A_train_80_with_embeddings.json"
FAISS_INDEX_PATH= "casebank_A_qwen_cls_cosine.index"
ID_MAP_PATH     = "casebank_A_qwen_cls_idmap.json"

def light_clean(x: str) -> str:
    if not x: return ""
    x = re.sub(r'(https?://\S+|www\.\S+)', ' ', x)
    x = x.replace('|',' ')
    x = re.sub(r'\s+', ' ', x).strip()
    return x

def ensure_post_casebank(item: dict, max_chars: int = 1200) -> str:
    """万一上一步没写入 post_casebank，这里兜底再构建一次。"""
    if "post_casebank" in item and item["post_casebank"].strip():
        return item["post_casebank"]

    base = light_clean(item.get("embed_text") or item.get("posts_cleaned") or item.get("posts") or "")
    sem  = light_clean(item.get("semantic_view",""))
    sen  = light_clean(item.get("sentiment_view",""))
    lin  = light_clean(item.get("linguistic_view",""))
    meta = []
    if sem: meta.append(f"[Semantic] {sem}")
    if sen: meta.append(f"[Sentiment] {sen}")
    if lin: meta.append(f"[Linguistic] {lin}")
    post_casebank = base if not meta else f"{base} " + " ".join(meta)
    if len(post_casebank) > max_chars:
        post_casebank = post_casebank[:max_chars].rstrip() + "…"
    return post_casebank

# === 加载 ===
with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

embs, id_map = [], {}
for idx, item in enumerate(raw):
    emb = np.array(item["embedding"], dtype=np.float32)
    embs.append(emb)
    id_map[idx] = {
        "post_casebank": ensure_post_casebank(item),
        "type": item.get("type", "")
    }

embs = np.vstack(embs).astype("float32")
faiss.normalize_L2(embs)                # 归一化使内积=余弦

dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embs)

faiss.write_index(index, FAISS_INDEX_PATH)
with open(ID_MAP_PATH, "w", encoding="utf-8") as f:
    json.dump(id_map, f, ensure_ascii=False, indent=2)

print("✅ 向量库已构建:", FAISS_INDEX_PATH)


In [ ]:
# sanity_check_and_search.py
import json, faiss, numpy as np

FAISS_INDEX_PATH = "casebank_A_qwen_cls_cosine.index"
ID_MAP_PATH      = "casebank_A_qwen_cls_idmap.json"
EMB_DATA_PATH    = "casebank_A_train_80_with_embeddings.json"  # 就是你建库时的那个 JSON

# 1) 加载索引和 id_map
index  = faiss.read_index(FAISS_INDEX_PATH)
with open(ID_MAP_PATH, "r", encoding="utf-8") as f:
    id_map = json.load(f)

print("index.ntotal =", index.ntotal, " | id_map size =", len(id_map))

# 2) 读取同一个 embeddings 数据文件，拿一个向量当 query
with open(EMB_DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

assert len(data) > 0, "embeddings 数据为空"
k = 10 if len(data) > 10 else 0                    # 选第 k 个样本
q = np.array(data[k]["embedding"], dtype=np.float32).reshape(1, -1)

# 如果你构建索引前做了 L2 归一化，这里最好再归一化一次（保险）
faiss.normalize_L2(q)

# 3) 检索 top-5
D, I = index.search(q, 5)

# 4) 打印结果
for s, idx in zip(D[0], I[0]):
    info = id_map[str(int(idx))]                   # id_map 的 key 是字符串
    print(f"{s:.4f}", info["type"], "|", info["post_casebank"][:80], "...")


In [ ]:
# encode_and_retrieve_topk.py
# -*- coding: utf-8 -*-
import json, faiss, numpy as np, torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from peft import PeftModel
import torch.nn.functional as F

# ========= 路径配置 =========
# 已建好的向量库（由 casebank_*_with_embeddings.json 构建）
FAISS_INDEX_PATH = "casebank_A_qwen_cls_cosine.index"
ID_MAP_PATH      = "casebank_A_qwen_cls_idmap.json"

# 待向量化并检索的测试集
TEST_INPUT          = "picked_balanced_around30.json"
TEST_WITH_EMB_OUT   = "A_test_with_embeddings_final1.json"
TOPK_RESULTS_OUT    = "A_test_top1_final1.json"

# Qwen + LoRA（与建库时完全一致）
BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_DIR = "mbti_lora_qwen1.5b-split_kaggle_ckpt"

TOPK = 1
MAX_LEN = 512
device = "cuda" if torch.cuda.is_available() else "cpu"

# ========= 加载索引 =========
index = faiss.read_index(FAISS_INDEX_PATH)
id_map = json.load(open(ID_MAP_PATH, "r", encoding="utf-8"))

# ========= 加载模型（与建库时一致）=========
tok = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

cfg = AutoConfig.from_pretrained(BASE_MODEL, trust_remote_code=True)
cfg.num_labels = 16

base = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, config=cfg, trust_remote_code=True,
    device_map={"": device}, low_cpu_mem_usage=True
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR, is_trainable=False).eval().to(device)

# ========= 工具函数 =========
def pick_text(item: dict) -> str:
    """按优先级选一个可用于编码的文本字段。按你的数据结构需要可调整顺序。"""
    for key in ["posts_cleaned_for_embedding", "embed_text", "post_casebank",
                "posts_cleaned", "posts", "text", "query_text"]:
        v = item.get(key)
        if isinstance(v, str) and v.strip():
            return v
    return ""

@torch.no_grad()
def encode_text_to_vec(text: str) -> np.ndarray:
    """
    用 masked mean pooling 提句向量（整段文本表征）并做 L2 归一化，
    与索引端 IndexFlatIP（内积≈余弦）一致。
    """
    ins = tok(text, return_tensors="pt", truncation=True, padding=True, max_length=MAX_LEN).to(device)
    out = model.base_model.model(**ins, output_hidden_states=True, use_cache=False, return_dict=True)
    last_hidden = out.hidden_states[-1]                 # (B, T, H)
    mask = ins["attention_mask"].unsqueeze(-1).float()  # (B, T, 1)

    masked = last_hidden * mask                         # zero out pads
    sum_hidden = masked.sum(dim=1)                      # (B, H)
    lengths = mask.sum(dim=1).clamp(min=1.0)            # (B, 1)
    vec = sum_hidden / lengths                          # (B, H)

    vec = F.normalize(vec, p=2, dim=1)                  # L2 归一化
    return vec.squeeze(0).float().cpu().numpy()         # (H,)

def search_topk(vec: np.ndarray, k=TOPK):
    D, I = index.search(vec.reshape(1, -1), k)
    return [{"score": float(s), **id_map[str(int(i))]} for s, i in zip(D[0], I[0])]

def filter_self_hits(query_text: str, hits: list, k: int):
    """可选：过滤与查询文本完全相同的返回，避免“命中自己”"""
    def norm(s): return " ".join((s or "").lower().split())
    qt = norm(query_text)
    keep = [h for h in hits if norm(h.get("post_casebank", "")) != qt]
    return keep[:k]

# ========= 读取测试集、编码、检索 =========
test = json.load(open(TEST_INPUT, "r", encoding="utf-8"))

with_emb, results = [], []
for it in tqdm(test, desc="Encode & retrieve"):
    text = pick_text(it)
    if not text:
        # 没有可用文本就记空
        it_out = dict(it); it_out["embedding"] = None
        with_emb.append(it_out)
        results.append({"type": it.get("type",""), "query_text": "", "topk_cases": []})
        continue

    emb = encode_text_to_vec(text)              # (H,)
    it_out = dict(it); it_out["embedding"] = emb.tolist()
    with_emb.append(it_out)

    topk = search_topk(emb, k=TOPK)
    topk = filter_self_hits(text, topk, TOPK)   # 可关掉：如果不担心命中自己
    results.append({
        "type": it.get("type",""),
        "query_text": text,
        "topk_cases": topk
    })

# ========= 落盘 =========
json.dump(with_emb, open(TEST_WITH_EMB_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)
json.dump(results, open(TOPK_RESULTS_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

print("✅ 向量化完成：", TEST_WITH_EMB_OUT)
print("✅ TopK 检索完成：", TOPK_RESULTS_OUT)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import json
from tqdm import tqdm

# ===== 模型配置 =====
MODEL_NAME = "microsoft/deberta-v3-base"
MODEL_PATH = "best_fem_deberta.pt"
USE_CLS = True
MIN_WORDS = 5 # 最小词数要求

# ===== 加载模型和 tokenizer =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
state_dict = torch.load(MODEL_PATH, map_location=torch.device("cpu"))
model.load_state_dict(state_dict, strict=False)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ===== 提取嵌入向量函数 =====
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        if USE_CLS:
            return outputs.last_hidden_state[:, 0, :].squeeze().cpu().tolist()
        else:
            return torch.mean(outputs.last_hidden_state, dim=1).squeeze().cpu().tolist()

# ===== 读取原始 casebank 数据 =====
with open("casebank_mbti.json", "r", encoding="utf-8") as f:
    data = json.load(f)

output = []
kept_count = 0
filtered_count = 0

for item in tqdm(data, desc="🔍 Extracting embeddings"):
    post_text = item.get("post_casebank", "").strip()
    mbti_type = item.get("type", "")

    # ===== 过滤短文本 =====
    word_count = len(post_text.split())
    if word_count < MIN_WORDS:
        filtered_count += 1
        continue

    emb = get_embedding(post_text)
    output.append({
        "embedding": emb,
        "post_casebank": post_text,
        "type": mbti_type
    })
    kept_count += 1

# ===== 保存 =====
output_file = "mbti_embeddings_from_custom_model.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

# ===== 打印统计结果 =====
print("✅ 完成！结果已保存至：", output_file)
print(f"📊 共处理样本数：{len(data)}")
print(f"✅ 保留样本数：{kept_count}")
print(f"🚫 筛除过短样本数：{filtered_count}（<{MIN_WORDS}词）")


In [ ]:
import json
import faiss
import numpy as np

# ========== 配置 ==========
DATA_PATH = "mbti_embeddings_from_custom_model.json"
FAISS_INDEX_PATH = "mbti_faiss_cosine.index"
ID_MAP_PATH = "mbti_idmap_cosine.json"

# ========== 加载嵌入 ==========
with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

embeddings = []
id_map = {}

for idx, item in enumerate(raw_data):
    emb = np.array(item["embedding"], dtype=np.float32)
    embeddings.append(emb)
    id_map[idx] = {
        "post_casebank": item["post_casebank"],
        "type": item["type"]
    }

# ========== 归一化 + 构建余弦相似度索引 ==========
embeddings = np.vstack(embeddings)
faiss.normalize_L2(embeddings)  # 👈 归一化后内积就等效于余弦相似度

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # 使用内积（归一化后等价于余弦）
index.add(embeddings)

# ========== 保存 ==========
faiss.write_index(index, FAISS_INDEX_PATH)
with open(ID_MAP_PATH, "w", encoding="utf-8") as f:
    json.dump(id_map, f, ensure_ascii=False, indent=2)

print("✅ 向量数据库（Cosine 相似度）构建完毕！")


In [ ]:
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

# 如果第一次运行，需要取消注释：
# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# ✅ 嵌入模型清洗：标准 NLP 清洗
def clean_for_embedding(text):
    text = text.lower()
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = text.replace('|', ' ')
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = nltk.word_tokenize(text)
    cleaned = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return ' '.join(cleaned)

# ✅ 大模型清洗：只去 URL 和 |
def clean_for_llm(text):
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = text.replace('|', ' ')
    return text.strip()

# 主处理函数
def preprocess_dual_clean(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    cleaned_data = []
    for item in tqdm(raw_data, desc="Cleaning Dual Versions"):
        post = item["posts"]
        cleaned_data.append({
            "type": item["type"],
            "posts": post,
            "posts_cleaned_for_embedding": clean_for_embedding(post),
            "posts_cleaned_for_llm": clean_for_llm(post)
        })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

    print(f"✅ 双重清洗完成，已保存至 {output_file}")


In [ ]:
preprocess_dual_clean("filtered_processed_comments_cleaned_random50.json", "pandora_cleaned.json")


In [ ]:
#验证数据做embedding
import json
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

MODEL_NAME = "microsoft/deberta-v3-base"
MODEL_PATH = "best_fem_deberta.pt"
USE_CLS = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
state_dict = torch.load(MODEL_PATH, map_location=torch.device("cpu"))
model.load_state_dict(state_dict, strict=False)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

# === 读取清洗后的验证集 ===
with open("pandora_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)


results = []
for item in tqdm(data, desc="Embedding"):
    emb = get_embedding(item["posts_cleaned_for_embedding"])
    results.append({
        "type": item["type"],
        "embedding": emb.tolist(),
        "posts_cleaned_for_llm": item["posts_cleaned_for_llm"],
        "posts_cleaned_for_embedding": item["posts_cleaned_for_embedding"]
    })

with open("pandora_with_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ 向量化完成！")


In [ ]:
#对验证数据进行向量相似度检索（Top-K）并准备给 LLM 使用的对话上下文结构

import json
import faiss
import numpy as np

# 读取 FAISS 和 ID 映射
index = faiss.read_index("mbti_faiss.index")
with open("mbti_idmap.json", "r", encoding="utf-8") as f:
    id_map = json.load(f)

def retrieve_topk(query_embedding, k=5):
    query_embedding = np.array(query_embedding).astype(np.float32).reshape(1, -1)
    D, I = index.search(query_embedding, k)
    return [id_map[str(i)] for i in I[0]]

# 读取验证集带 embedding 的数据
with open("pandora_with_embeddings.json", "r", encoding="utf-8") as f:
    verification_data = json.load(f)

retrieved_results = []

for item in verification_data:
    query_emb = item["embedding"]
    topk_cases = retrieve_topk(query_emb, k=3)
    retrieved_results.append({
        "type": item["type"],
        "posts_cleaned_for_llm": item["posts_cleaned_for_llm"],
        "posts_cleaned_for_embedding": item["posts_cleaned_for_embedding"],
        "topk_cases": topk_cases
    })

with open("retrieved_results_for_llm_pandora.json", "w", encoding="utf-8") as f:
    json.dump(retrieved_results, f, ensure_ascii=False, indent=2)

print("✅ 检索完成！结果保存在 retrieved_results_for_llm_pandora.json")
 

In [ ]:
import json

path = "retrieved_results_for_llm_pandora.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)  # 小心⚠️：大文件容易爆内存

# 只看前 5 条
for i, item in enumerate(data[:5]):
    print(f"\n=== 第 {i+1} 条样本 ===")
    print("Type:", item["type"])
    print("Query Post:", item["posts_cleaned_for_llm"][:300], "...")
    print("Top-K types:", [x["type"] for x in item["topk_cases"]])


In [ ]:
from collections import defaultdict, Counter
import json, re, requests
from tqdm import tqdm

# === Extract final MBTI type from LLM output ===
def extract_final_type(text):
    match = re.search(r'(Final Type|最终类型)[:：]?\s*([IEie][NSns][FTft][JPjp])\b', text)
    if match:
        return match.group(2).upper()
    return "UNKNOWN"

# === Build the prompt ===
def build_prompt(user_post, topk_cases):
    examples_text = "\n\n".join([
        f"[Reference Example {i+1}]\nPost Content: {ex['post_casebank']}\nMBTI Type: {ex['type']}" 
        for i, ex in enumerate(topk_cases)
    ])
    prompt = f"""
You are an expert in MBTI personality typing and linguistic style analysis.

[User Post]
{user_post}

[Reference Examples]
{examples_text}

---
1. Final Type: ______
2. Analyze the writing style, tone, logicality, and emotionality.
3. Compare it with each reference example and explain similarities.
4. Conclude with the most likely MBTI type.
"""
    return prompt

# === LLM API wrapper ===
class LLM_API_Wrapper:
    def __init__(self, model, api_key):
        self.model = model
        self.api_key = api_key

    def call_api(self, prompt_text: str, n: int = 1):
        url = ""
        payload = {
            "model": self.model,
            "temperature": 0.8,
            "n": n,
            "messages": [{"role": "user", "content": prompt_text}],
            "max_completion_tokens": 512,
            "stream": False
        }
        headers = {"Content-Type": "application/json","Authorization": f"Bearer {self.api_key}"}
        try:
            r = requests.post(url, headers=headers, json=payload, timeout=60)
            r.raise_for_status()
            result = r.json()
            return [c["message"]["content"] for c in result.get("choices", [])]
        except Exception as e:
            print("❌ Request failed:", e)
            return []

# === Call until we collect enough valid responses ===
def robust_llm_call(llm, prompt, vote_times=40, n_valid=20, max_total_requests=20):
    all_valid_responses, total_requests = [], 0
    while len(all_valid_responses) < n_valid and total_requests < max_total_requests:
        total_requests += 1
        new_responses = llm.call_api(prompt, n=vote_times)
        valid = [r for r in new_responses if extract_final_type(r) != "UNKNOWN"]
        all_valid_responses.extend(valid)
        print(f"🔄 Collected {len(all_valid_responses)} valid responses...")
    return all_valid_responses[:n_valid]

# === Main function: CoT-SC majority voting ===
def evaluate_with_voting(input_file, output_file, llm: LLM_API_Wrapper, vote_times=40, n_valid=20):
    with open(input_file, "r", encoding="utf-8") as f:
        all_inputs = json.load(f)

    outputs, correct, total = [], 0, 0
    type_correct, type_total = defaultdict(int), defaultdict(int)

    for item in tqdm(all_inputs, desc="🔁 Running CoT-SC voting"):
        prompt = build_prompt(item["query_text"], item["topk_cases"])
        responses = robust_llm_call(llm, prompt, vote_times, n_valid)
        if responses:
            predicted_types = [extract_final_type(r) for r in responses]
            majority_vote = Counter(predicted_types).most_common(1)[0][0]
            ground_truth = item.get("type", "UNKNOWN").upper()
            is_correct = majority_vote == ground_truth
            outputs.append({
                "query_text": item["query_text"],
                "ground_truth": ground_truth,
                "final_prediction": majority_vote,
                "correct": is_correct,
                "llm_responses": responses
            })
            if ground_truth != "UNKNOWN":
                total += 1; type_total[ground_truth]+=1
                if is_correct: correct+=1; type_correct[ground_truth]+=1
        else:
            outputs.append({"query_text": item["query_text"],"final_prediction":"ERROR","correct":False})
    json.dump(outputs, open(output_file,"w",encoding="utf-8"),ensure_ascii=False,indent=2)
    print(f"✅ Voting finished. Results saved to {output_file}")
    if total>0:
        print(f"🎯 Accuracy: {correct}/{total} = {correct/total:.4f}")


In [ ]:
from collections import Counter
import json

with open("test对应的原始数据.json", "r", encoding="utf-8") as f:
    data = json.load(f)

cnt = Counter([d.get("type", "UNKNOWN").upper() for d in data])
print(cnt)


In [ ]:
import json
from collections import defaultdict

INPUT_FILE  = "QwenBase_test_topk3.json"     # 输入文件
OUTPUT_FILE = "QwenBase_test_topk3_clip32.json"  # 输出文件
MAX_PER_TYPE = 32

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

buckets = defaultdict(list)
for item in data:
    t = item.get("type", "UNKNOWN").upper()
    if len(buckets[t]) < MAX_PER_TYPE:
        buckets[t].append(item)

# 拼回列表
clipped = []
for t, items in buckets.items():
    clipped.extend(items)
    print(f"{t}: {len(items)} kept")

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(clipped, f, ensure_ascii=False, indent=2)

print(f"✅ Done! Saved to {OUTPUT_FILE}, total {len(clipped)} items")


In [ ]:
llm = LLM_API_Wrapper(
    model="gpt-4o-mini",        # or your deployed model name
    api_key="" # replace with your actual key
)

evaluate_with_voting(
    input_file="B_test_20_topk_clip50.json",         # your TopK retrieved file
    output_file="B_test_20_clip50_vote_results.json",# output file
    llm=llm,
    vote_times=5,   # how many responses per request
    n_valid=5       # how many valid responses needed
)


In [ ]:
import json
from collections import defaultdict

def evaluate_mbti_accuracy(result_file):
    with open(result_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 四个维度统计
    dim_names = ["IE", "NS", "TF", "JP"]
    dim_correct = [0, 0, 0, 0]
    dim_total = [0, 0, 0, 0]

    # 完整四字母
    total, correct = 0, 0

    # 每种类型的统计
    type_total = defaultdict(int)
    type_correct = defaultdict(int)

    for item in data:
        gt = item["ground_truth"].upper()
        pred = item["final_prediction"].upper()

        if len(gt) == 4 and len(pred) == 4:
            total += 1
            type_total[gt] += 1

            if gt == pred:
                correct += 1
                type_correct[gt] += 1

            # 四个维度逐位比较
            for i in range(4):
                dim_total[i] += 1
                if gt[i] == pred[i]:
                    dim_correct[i] += 1

    # 输出结果
    print("=== 四个维度准确率 ===")
    for name, c, t in zip(dim_names, dim_correct, dim_total):
        acc = c / t if t > 0 else 0
        print(f"{name}: {c}/{t} = {acc:.4f}")

    print("\n=== 四字母完全匹配准确率 ===")
    print(f"Overall: {correct}/{total} = {correct/total:.4f}")

    print("\n=== 每个类型的准确率 ===")
    all_types = [
        "INTJ","INTP","ENTJ","ENTP",
        "INFJ","INFP","ENFJ","ENFP",
        "ISTJ","ISFJ","ESTJ","ESFJ",
        "ISTP","ISFP","ESTP","ESFP"
    ]
    for t in all_types:
        c, tt = type_correct[t], type_total[t]
        acc = c / tt if tt > 0 else 0
        print(f"{t}: {c}/{tt} = {acc:.4f}")


# 调用
evaluate_mbti_accuracy("B_test_20_clip50_vote_results.json")


In [ ]:
from collections import defaultdict, Counter
import json
import re
from tqdm import tqdm
import requests

# === 提取最终 MBTI 类型 ===
def extract_final_type(text):
    # 兼容“Final Type”和“最终类型”两种写法
    match = re.search(r'(Final Type|最终类型)[:：]?\s*([IEie][NSns][FTft][JPjp])\b', text)
    if match:
        return match.group(2).upper()
    return "UNKNOWN"

# === 构造 Prompt ===
def build_prompt(user_post, topk_cases):
    examples_text = "\n\n".join([
        f"[Reference Example {i+1}]\nPost Content: {ex['post_casebank']}\nMBTI Type: {ex['type']}" 
        for i, ex in enumerate(topk_cases)
    ])

    prompt = f"""
You are an expert in MBTI personality typing and linguistic style analysis.

Your task is to infer the most likely MBTI type of a user's post **by analyzing their writing style** — such as tone, phrasing, language complexity, emotionality, and personality cues — and comparing it to several known examples labeled with MBTI types.

**You must treat the reference examples as correct and trustworthy.**
Do not ignore them — they provide strong clues about how different types express themselves.

---

[User Post]
{user_post}

[Reference Examples]
{examples_text}

---

Now follow these steps:
1.Final Type: ______

2. **Analyze** the user's post: What is its tone? Abstract or concrete? Emotional or logical? Structured or scattered?

3. **Compare** it with each reference example: Which ones are stylistically most similar to the user? Provide reasons.

4. **Conclude** with the MBTI type that best matches the user post, based on the closest stylistic match.
"""
    return prompt

# === 封装 LLM API 接口 ===
class LLM_API_Wrapper:
    def __init__(self, model, api_key):
        self.model = model
        self.api_key = api_key

    def call_api(self, prompt_text: str, n: int = 1):
        url = ""
        payload = {
            "model": self.model,
            "temperature": 0.8,
            "n": n,
            "messages": [{"role": "user", "content": prompt_text}],
            "modalities": ["text"],
            "response_format": {"type": "text"},
            "max_completion_tokens": 512,
            "stream": False
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        try:
            response = requests.post(url, headers=headers, json=payload, timeout=60)
            response.raise_for_status()
            result = response.json()
            if "choices" in result:
                return [c["message"]["content"] for c in result["choices"]]
            else:
                print("⚠️ Missing 'choices' in response.")
                return []
        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")
            return []

# === 循环直到获得 n_valid 条有效响应 ===
def robust_llm_call(llm, prompt, vote_times=40, n_valid=20, max_total_requests=20):
    all_valid_responses = []
    total_requests = 0

    while len(all_valid_responses) < n_valid:
        total_requests += 1
        if total_requests > max_total_requests:
            print("⚠️ 请求次数过多，跳过该样本")
            return []

        new_responses = llm.call_api(prompt, n=vote_times)
        if not new_responses:
            continue

        valid = [r for r in new_responses if extract_final_type(r) != "UNKNOWN"]
        all_valid_responses.extend(valid)

        print(f"🔄 已累计 {len(all_valid_responses)} 条有效回答...")

    return all_valid_responses[:n_valid]

# === 主函数：CoT-SC 多轮投票预测 ===
def evaluate_with_voting(input_file, output_file, llm: LLM_API_Wrapper, vote_times=40, n_valid=20):
    with open(input_file, "r", encoding="utf-8") as f:
        all_inputs = json.load(f)

    outputs = []
    correct = 0
    total = 0
    type_correct = defaultdict(int)
    type_total = defaultdict(int)

    for item in tqdm(all_inputs, desc="🔁 正在进行 CoT-SC 预测"):
        prompt = build_prompt(item["query_post"], item["topk_cases"])
        responses = robust_llm_call(llm, prompt, vote_times, n_valid)

        if responses:
            predicted_types = [extract_final_type(r) for r in responses]
            majority_vote = Counter(predicted_types).most_common(1)[0][0]
            ground_truth = item.get("type", "UNKNOWN").upper()
            is_correct = majority_vote == ground_truth

            outputs.append({
                "query_post": item["query_post"],
                "ground_truth": ground_truth,
                "llm_responses": responses,
                "predicted_types": predicted_types,
                "final_prediction": majority_vote,
                "correct": is_correct,
                "prompt": prompt
            })

            if ground_truth != "UNKNOWN":
                total += 1
                type_total[ground_truth] += 1
                if is_correct:
                    correct += 1
                    type_correct[ground_truth] += 1
        else:
            outputs.append({
                "query_post": item["query_post"],
                "ground_truth": item.get("type", "UNKNOWN"),
                "llm_responses": [],
                "predicted_types": [],
                "final_prediction": "ERROR",
                "correct": False,
                "prompt": prompt
            })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(outputs, f, ensure_ascii=False, indent=2)

    print(f"\n✅ 投票预测完成，结果保存至 {output_file}")
    if total > 0:
        acc = correct / total
        print(f"\n🎯 总体准确率：{acc:.4f}（{correct} / {total}）")
        print("\n📊 各类型准确率：")
        for mbti_type in sorted(type_total.keys()):
            total_count = type_total[mbti_type]
            correct_count = type_correct[mbti_type]
            acc_type = correct_count / total_count
            print(f"{mbti_type}: {acc_type:.2f} ({correct_count} / {total_count})")
    else:
        print("⚠️ 没有包含 ground_truth 的数据，无法计算准确率。")



In [ ]:
import json
from collections import Counter

with open("retrieved_results_for_llm2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 提取所有的类型
types = [item["type"].upper() for item in data if "type" in item]

# 统计数量
type_counts = Counter(types)

# 输出每种类型的样本数
for mbti_type, count in sorted(type_counts.items()):
    print(f"{mbti_type}: {count} samples")

In [ ]:
llm = LLM_API_Wrapper(model="gpt-4o-mini", api_key="")

evaluate_with_voting(
    input_file="retrieved_results_for_llm2.json",
    output_file="voting_eval_results1.json",
    llm=llm,
    vote_times=5,
    n_valid=5
)


In [ ]:
llm = LLM_API_Wrapper(model="gpt-4o", api_key="")

evaluate_with_voting(
    input_file="retrieved_results_for_llm_cosine.json",
    output_file="voting_eval_results_cosine.json",
    llm=llm,
    vote_times=10,
    n_valid=5
)

In [ ]:
######  Retain
import json
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel

# ===== 1. 模型加载 =====
MODEL_NAME = "microsoft/deberta-v3-base"
MODEL_PATH = "best_fem_deberta.pt"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
state_dict = torch.load(MODEL_PATH, map_location=torch.device("cpu"))
model.load_state_dict(state_dict, strict=False)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ===== 2. 向量提取函数 =====
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

# ===== 3. 加载 LLM 输出结果（已包含预测类型）=====
with open("llm_outputs_result.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# ===== 4. 构建 Retain 样本并提取向量 =====
new_vectors = []
new_id_map = {}
new_data = []

for i, item in enumerate(results):
    query_post = item["query_post"]
    posts_cleaned = item.get("posts_cleaned", "")
    final_pred = item.get("final_prediction", "UNKNOWN")
    topk_cases = item.get("topk_cases", [])

    post_casebank_text = "|||".join([case["post_casebank"] for case in topk_cases])

    record = {
        "type": final_pred,
        "posts": query_post,
        "posts_cleaned": posts_cleaned,
        "post_casebank": post_casebank_text
    }

    emb = get_embedding(post_casebank_text)
    new_vectors.append(emb.astype(np.float32))
    new_id_map[str(i)] = record
    new_data.append(record)

# ===== 5. 加载原 FAISS index + id_map 并更新 =====
index = faiss.read_index("mbti_faiss.index")
with open("mbti_idmap.json", "r", encoding="utf-8") as f:
    id_map = json.load(f)

start_id = len(id_map)
for i, emb in enumerate(new_vectors):
    index.add(np.array([emb]))  # 添加新向量
    id_map[str(start_id + i)] = new_id_map[str(i)]

# ===== 6. 保存更新后的 index 和 id_map =====
faiss.write_index(index, "mbti_faiss.index")
with open("mbti_idmap.json", "w", encoding="utf-8") as f:
    json.dump(id_map, f, ensure_ascii=False, indent=2)

print(f"✅ Retain 完成：已添加 {len(new_vectors)} 条样本至向量数据库。")


In [ ]:
# 初始化你的 LLM 接口封装（替换为你的真实 API KEY）
llm = LLM_API_Wrapper(model="gpt-4o", api_key="")

# 运行评估：输入为你构造的 TopK 检索结果
evaluate_with_voting(
    input_file="retrieved_results_for_llm.json",
    output_file="voting_eval_results.json",
    llm=llm,
    vote_times=5  # 建议先用 5 轮测试，正式评估用 20 或 40
)


In [ ]:
# 初始化你的 LLM 接口封装（替换为你的真实 API KEY）
llm = LLM_API_Wrapper(model="gpt-4o", api_key="")

# 运行评估：输入为你构造的 TopK 检索结果
evaluate_with_voting(
    input_file="retrieved_results_for_llm.json",
    output_file="voting_eval_results.json",
    llm=llm,
    vote_times=5  # 建议先用 5 轮测试，正式评估用 20 或 40
)

In [ ]:
def run_single_query_pipeline(raw_text: str, llm: LLM_API_Wrapper, vote_times: int = 5, do_retain=True):
    # 1. 清洗
    posts_cleaned = clean_text(raw_text)

    # 2. 嵌入
    query_emb = get_embedding(posts_cleaned)

    # 3. 检索
    topk_cases = retrieve_topk(query_emb, k=5)

    # 4. Prompt 构造
    prompt = build_prompt(raw_text, topk_cases)

    # 5. LLM 推理
    responses = llm.call_api(prompt, n=vote_times)
    predicted_types = [extract_final_type(r) for r in responses]
    majority_vote = Counter(predicted_types).most_common(1)[0][0]

    result = {
        "query_post": raw_text,
        "posts_cleaned": posts_cleaned,
        "topk_cases": topk_cases,
        "llm_responses": responses,
        "predicted_types": predicted_types,
        "final_prediction": majority_vote,
        "prompt": prompt
    }

    # 6. RETAIN（可选）
    if do_retain:
        post_casebank_text = "|||".join([case["post_casebank"] for case in topk_cases])
        emb = get_embedding(post_casebank_text)

        index = faiss.read_index("mbti_faiss.index")
        with open("mbti_idmap.json", "r", encoding="utf-8") as f:
            id_map = json.load(f)

        new_id = str(len(id_map))
        index.add(np.array([emb.astype(np.float32)]))
        id_map[new_id] = {
            "post_casebank": post_casebank_text,
            "type": majority_vote
        }

        faiss.write_index(index, "mbti_faiss.index")
        with open("mbti_idmap.json", "w", encoding="utf-8") as f:
            json.dump(id_map, f, ensure_ascii=False, indent=2)

    return result


In [ ]:
def run_pipeline_from_json_file(json_path: str, llm: LLM_API_Wrapper, vote_times=5, retain=True):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    all_results = []
    print(f"\n📄 正在处理 JSON 文档，共 {len(data)} 条记录...\n")

    for idx, item in enumerate(data):
        text = item.get("posts", "").strip()
        if not text:
            continue

        print(f"🔁 第 {idx+1}/{len(data)} 条用户发言处理中...")

        result = run_single_query_pipeline(text, llm, vote_times=vote_times, do_retain=retain)

        # 可选：记录原始标签
        result["ground_truth"] = item.get("type", "UNKNOWN")
        all_results.append(result)

    output_path = f"mbti_predictions_from_{os.path.basename(json_path)}"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"\n✅ 全部完成！预测结果已保存至：{output_path}")
